
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_CGMGPT',
 'MODEL_TASK': 'Model_CGMGPT',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared '
                   'drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-WorkSpace'}


# Dataset

In [2]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

# TriggerCaseMethod

In [3]:
from recfldtkn.pipeline_model import get_Trigger_Cases, convert_TriggerCases_to_LearningCases

use_learning = True 
use_inference = not use_learning

######################################
TriggerCaseMethod = 'CGM5MinEntry'
cohort_label_list = [1]
Trigger2LearningMethods = [
    
    {'op':'Tag',    'Name': 'Tag_PttBasicWD'},
    {'op':'Filter', 'Name': 'Filter_PttBasicWD'},
    
    {'op':'CFQ',    'Name': 'CFQ_Bf24hCGMrn'},
    {'op':'TagCF',  'Name': 'TagCF_Bf24hCGMrn', 'CFQName': 'CFQ_Bf24hCGMrn'},
    
    {'op':'CFQ',    'Name': 'CFQ_Af2hCGMrn',    
     'type': 'learning-only'},
    {'op':'TagCF',  'Name': 'TagCF_Af2hCGMrn',  'CFQName': 'CFQ_Af2hCGMrn',  
     'type': 'learning-only',},
    
    {'op':'Filter', 'Name': 'Filter_BfCGMgeq280'},
    {'op':'Filter', 'Name': 'Filter_AfCGMgeq24',  
     'type': 'learning-only'},
    
    {'op':'CFQ',    'Name': 'CFQ_Bf1mMEDALrn',    
     'type': 'learning-only'},
    {'op':'TagCF',  'Name': 'TagCF_Bf1mMEDALrn', 'CFQName': 'CFQ_Bf1mMEDALrn', 
     'type': 'learning-only'},

]
######################################

In [4]:
use_inference

False

In [5]:
from recfldtkn.loadtools import fetch_casetag_tools, fetch_casefilter_tools, load_module_variables

def convert_TriggerCases_to_LearningCases(df_case, 
                                          cohort_label_list,
                                          Trigger2LearningMethods, 
                                          base_config, 
                                          use_inference):
    
    CFQ_to_CaseFeatInfo = {}

    if use_inference == True:
        Trigger2LearningMethods = [i for i in Trigger2LearningMethods if i.get('type', None) != 'learning-only']
       
    # print(Trigger2LearningMethods)
    SPACE = base_config['SPACE']
    for method in Trigger2LearningMethods:
        if method['op'] == 'Tag':
            name = method['Name']
            logger.info(f'CaseTag: {name}')
            CaseTag_Tools = fetch_casetag_tools(name, SPACE)

            subgroup_columns = CaseTag_Tools['subgroup_columns']
            if 'InfoRecName' in CaseTag_Tools:
                InfoRecName = CaseTag_Tools['InfoRecName']
                ds_info, _ = load_ds_rec_and_info(InfoRecName, base_config, cohort_label_list)
            else:
                ds_info = None

            fn_case_tagging = CaseTag_Tools['fn_case_tagging']
            df_case = fn_case_tagging(df_case, ds_info, subgroup_columns, base_config)

        elif method['op'] == 'Filter':
            name = method['Name']
            logger.info(f'CaseFilter: {name}')
            CaseFilter_Tools = fetch_casefilter_tools(name, SPACE)
            fn_case_filtering = CaseFilter_Tools['fn_case_filtering']
            
            logger.info(f'Before Filter: {df_case.shape}')
            df_case = fn_case_filtering(df_case)
            logger.info(f'After Filter: {df_case.shape}')

        elif method['op'] == 'CFQ':
            name = method['Name']
            pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{name}.py')
            module = load_module_variables(pypath)
            fn_casefeat_querying = module.fn_casefeat_querying
            df_case, CaseFeatInfo = fn_casefeat_querying(df_case, base_config)
            # CaseFeatName = CaseFeatInfo['CaseFeatName']
            CFQ_to_CaseFeatInfo[name] = CaseFeatInfo

        elif method['op'] == 'TagCF':
            name = method['Name']
            pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{name}.py')
            module = load_module_variables(pypath)
            fn_case_tagging_on_casefeat = module.fn_case_tagging_on_casefeat
            
            CFQName = method['CFQName']
            CFQ_to_CaseFeatInfo = CFQ_to_CaseFeatInfo[CFQName]
            df_case = fn_case_tagging_on_casefeat(df_case, CaseFeatInfo)

        else:
            raise ValueError(f'Unknown method: {method}')

    return df_case

In [6]:
RecName_to_dsRec = {}
RecName_to_dsRecInfo = {}

df_case = get_Trigger_Cases(TriggerCaseMethod, 
                            cohort_label_list, 
                            base_config, 
                            SPACE, 
                            RecName_to_dsRec, 
                            RecName_to_dsRecInfo)

#####################
df_case = df_case.sample(1000, random_state=42)
#####################

logger.info(f'Before: {df_case.shape}')
df_case = convert_TriggerCases_to_LearningCases(df_case, 
                                                cohort_label_list,
                                                Trigger2LearningMethods, 
                                                base_config, 
                                                use_inference)
logger.info(f'After: {df_case.shape}')

[INFO:2024-03-03 22:25:45,045:(3752040664.py@15 __main__)]: Before: (1000, 2)
[INFO:2024-03-03 22:25:45,046:(2962776366.py@19 __main__)]: CaseTag: Tag_PttBasicWD
[INFO:2024-03-03 22:25:45,061:(2962776366.py@34 __main__)]: CaseFilter: Filter_PttBasicWD
[INFO:2024-03-03 22:25:45,062:(2962776366.py@38 __main__)]: Before Filter: (1000, 7)
[INFO:2024-03-03 22:25:45,063:(2962776366.py@40 __main__)]: After Filter: (974, 7)
[INFO:2024-03-03 22:25:47,199:(observer.py@892 recfldtkn.observer)]: provided casefeat_ids num: 974
[INFO:2024-03-03 22:25:47,209:(observer.py@582 recfldtkn.observer)]: provided caseobs_ids num: 974


Map:   0%|          | 0/974 [00:00<?, ? examples/s]

[INFO:2024-03-03 22:25:51,189:(observer.py@892 recfldtkn.observer)]: provided casefeat_ids num: 974
[INFO:2024-03-03 22:25:51,198:(observer.py@582 recfldtkn.observer)]: provided caseobs_ids num: 974


Map:   0%|          | 0/974 [00:00<?, ? examples/s]

[INFO:2024-03-03 22:25:52,731:(2962776366.py@34 __main__)]: CaseFilter: Filter_BfCGMgeq280
[INFO:2024-03-03 22:25:52,732:(2962776366.py@38 __main__)]: Before Filter: (974, 13)
[INFO:2024-03-03 22:25:52,734:(2962776366.py@40 __main__)]: After Filter: (760, 13)
[INFO:2024-03-03 22:25:52,734:(2962776366.py@34 __main__)]: CaseFilter: Filter_AfCGMgeq24
[INFO:2024-03-03 22:25:52,735:(2962776366.py@38 __main__)]: Before Filter: (760, 13)
[INFO:2024-03-03 22:25:52,737:(2962776366.py@40 __main__)]: After Filter: (729, 13)
[INFO:2024-03-03 22:25:52,748:(loadtools.py@146 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT/2-RawData2023_CVSTDCAug/MedPres_data ...
[INFO:2024-03-03 22:25:52,748:(loadtools.py@146 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT/3-RawData2023_CVSDeRxAug/MedPres_data ...
[INFO:2024-03-03 22:25:52,770:(loadtools.py@146 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT/2-RawData2023_CVSTDCAug/MedAdmin_data ...
[INFO:2024-03-03 22:25:52,771:(load

Map:   0%|          | 0/729 [00:00<?, ? examples/s]

[INFO:2024-03-03 22:25:55,652:(3752040664.py@21 __main__)]: After: (729, 29)


In [7]:
df_case

,PID,ObsDT,Gender,YearOfBirth,DiseaseType,cohort,Age,cf.InputCatCOs_co.OQTPe73SNH:input_ids,cf.InputCatCOs_co.OQTPe73SNH:input_wgts,BfCGMrn,...,BfLessonProgRN,BfFoodRecRN,BfCarbRN,BfExerciseRecRN,BfSleepRecRN,BfStepRecRN,BfBPRN,BfWeighURN,BfLabTestRestRN,MEDALrn
0,1002319,2021-08-16 02:10:00,1,1976,2,C1,45,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001911,2021-06-21 11:20:00,1,1994,2,C1,27,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000987,2021-11-23 19:05:00,2,1957,1,C1,64,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1001452,2021-12-01 01:35:00,2,1974,2,C1,47,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,40.0,0.0,10.0,0.0,0.0,0.0,50.0
4,1001088,2021-08-05 21:05:00,2,1953,1,C1,68,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,1001942,2021-11-25 16:05:00,2,1974,2,C1,47,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,22.0,97.0,15.0,6.0,0.0,0.0,140.0
725,1002840,2021-12-07 20:10:00,2,1957,2,C1,64,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
726,1000024,2021-01-26 05:00:00,1,1990,1,C1,31,"[1, 2]","[285.0, 1440.0]",285.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
727,1004164,2021-11-27 09:35:00,1,1981,1,C1,40,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Split

In [8]:
df_case # df_case_learning

,PID,ObsDT,Gender,YearOfBirth,DiseaseType,cohort,Age,cf.InputCatCOs_co.OQTPe73SNH:input_ids,cf.InputCatCOs_co.OQTPe73SNH:input_wgts,BfCGMrn,...,BfLessonProgRN,BfFoodRecRN,BfCarbRN,BfExerciseRecRN,BfSleepRecRN,BfStepRecRN,BfBPRN,BfWeighURN,BfLabTestRestRN,MEDALrn
0,1002319,2021-08-16 02:10:00,1,1976,2,C1,45,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001911,2021-06-21 11:20:00,1,1994,2,C1,27,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000987,2021-11-23 19:05:00,2,1957,1,C1,64,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1001452,2021-12-01 01:35:00,2,1974,2,C1,47,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,40.0,0.0,10.0,0.0,0.0,0.0,50.0
4,1001088,2021-08-05 21:05:00,2,1953,1,C1,68,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,1001942,2021-11-25 16:05:00,2,1974,2,C1,47,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,22.0,97.0,15.0,6.0,0.0,0.0,140.0
725,1002840,2021-12-07 20:10:00,2,1957,2,C1,64,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
726,1000024,2021-01-26 05:00:00,1,1990,1,C1,31,"[1, 2]","[285.0, 1440.0]",285.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
727,1004164,2021-11-27 09:35:00,1,1981,1,C1,40,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from recfldtkn.pipeline_model import generate_random_tags, assign_caseSplitTag_to_dsCaseLearning

##############
RANDOM_SEED = 42
downsample_ratio = 0.1 # 1 (don't drop any case), 0.1 (drop 90% of cases of one patient).
out_ratio = 0.1 # hold-out patients
test_ratio = 'tail0.1' #  '2023.10.15'#  # '0.1'
valid_ratio = 0.1 
##############

SplitMethod = f'rs{RANDOM_SEED}-ds{downsample_ratio}-out{out_ratio}ts{test_ratio}vd{valid_ratio}' 
print(SplitMethod)

rs42-ds0.1-out0.1tstail0.1vd0.1


In [10]:
logger.info('Generate Random Tags for Downsample, In/Out, and Train/Test/Valid Split')
logger.info(f'SplitMethod: {SplitMethod}')
logger.info(f'Before Split: {df_case.shape}')
df_case = assign_caseSplitTag_to_dsCaseLearning(df_case, 
                                                RANDOM_SEED, 
                                                downsample_ratio, out_ratio, 
                                                test_ratio, valid_ratio)
logger.info(f'After Split: {df_case.shape}')
df_case.head()

[INFO:2024-03-03 22:28:00,202:(3087417902.py@1 __main__)]: Generate Random Tags for Downsample, In/Out, and Train/Test/Valid Split
[INFO:2024-03-03 22:28:00,203:(3087417902.py@2 __main__)]: SplitMethod: rs42-ds0.1-out0.1tstail0.1vd0.1
[INFO:2024-03-03 22:28:00,203:(3087417902.py@3 __main__)]: Before Split: (729, 29)
[INFO:2024-03-03 22:28:00,224:(3087417902.py@8 __main__)]: After Split: (83, 34)


,PID,ObsDT,Gender,YearOfBirth,DiseaseType,cohort,Age,cf.InputCatCOs_co.OQTPe73SNH:input_ids,cf.InputCatCOs_co.OQTPe73SNH:input_wgts,BfCGMrn,...,BfStepRecRN,BfBPRN,BfWeighURN,BfLabTestRestRN,MEDALrn,Out,In,Test,Valid,Train
0,1002083,2021-11-18 02:10:00,2,1954,1,C1,67,"[1, 2]","[289.0, 1440.0]",289.0,...,30.0,0.0,0.0,0.0,67.0,False,True,False,True,False
1,1000821,2021-06-26 03:25:00,2,1979,1,C1,42,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,False,True,False,False,True
2,1001314,2021-09-04 03:25:00,1,1983,1,C1,38,"[1, 2]","[287.0, 1440.0]",287.0,...,0.0,0.0,0.0,0.0,0.0,False,True,False,False,True
3,1003467,2021-12-09 19:10:00,2,1952,1,C1,69,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,False,True,False,False,True
4,1001717,2021-10-15 07:50:00,2,1946,2,C1,75,"[1, 2]","[289.0, 1440.0]",289.0,...,0.0,0.0,0.0,0.0,0.0,False,True,False,False,True


In [12]:
print(f'\ntotal---> recnum {len(df_case)}')
for i in ['In', 'Out', 'Train', 'Valid', 'Test']:
    print(i, df_case[i].mean())

print('Valid/(Train+Test):', df_case['Valid'].sum() / (df_case['Train'].sum() + df_case['Valid'].sum()))

# subtype_list = ['patient_gender', 'patient_age_bucket', 'patient_zipcode_3']
subtype_list = ['Gender', 'DiseaseType']
for subtype in subtype_list:
    for subname, df_sub in df_case.groupby(subtype):
        print(f'\n{subname}---> recnum {len(df_sub)}')
        for i in ['In', 'Out', 'Train', 'Valid', 'Test']:
            print(i, df_sub[i].mean())
        print('Valid/(Train+Test):', df_sub['Valid'].sum() / (df_sub['Train'].sum() + df_sub['Valid'].sum()))


total---> recnum 83
In 0.9036144578313253
Out 0.0963855421686747
Train 0.8795180722891566
Valid 0.12048192771084337
Test 0.0
Valid/(Train+Test): 0.12048192771084337

1---> recnum 38
In 0.9473684210526315
Out 0.05263157894736842
Train 0.8947368421052632
Valid 0.10526315789473684
Test 0.0
Valid/(Train+Test): 0.10526315789473684

2---> recnum 45
In 0.8666666666666667
Out 0.13333333333333333
Train 0.8666666666666667
Valid 0.13333333333333333
Test 0.0
Valid/(Train+Test): 0.13333333333333333

1---> recnum 44
In 0.9772727272727273
Out 0.022727272727272728
Train 0.9090909090909091
Valid 0.09090909090909091
Test 0.0
Valid/(Train+Test): 0.09090909090909091

2---> recnum 39
In 0.8205128205128205
Out 0.1794871794871795
Train 0.8461538461538461
Valid 0.15384615384615385
Test 0.0
Valid/(Train+Test): 0.15384615384615385
